In [539]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

# from src import const
from pathlib import Path
Path.cwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


PosixPath('/Users/adamzh0u/170_BI/opt_travel_cost')

## load data

In [325]:
import pdfplumber

with pdfplumber.open("data/World Tour Complete Pack.pdf") as pdf:

    time = pdf.pages[2].extract_table()
    cost = pdf.pages[3].extract_table()
    flight_info = pdf.pages[4].extract_table()
    venue_info = pdf.pages[5].extract_table()
    # hotel_info = pdf.pages[7].images
    change_rate = pdf.pages[8].extract_table()

In [360]:
# thank you gpt
hotel_info = {
    "Warsaw": 80, 
    "Istanbul": 85,
    "Lagos": 90,
    "Delhi": 95,
    "BuenosAires": 100,
    "RiodeJaneiro": 100,
    "CapeTown": 100,
    "MexicoCity": 105,
    "Beijing": 110,
    "Bangkok": 110,
    "Lisbon": 120,
    "Seoul": 140,
    "Dubai": 145,
    "Rome": 150,
    "Berlin": 160,
    "Amsterdam": 170,
    "Sydney": 175,
    "Helsinki": 180,
    "Paris": 180,
    "Barcelona": 185,
    "Singapore": 195,
    "London": 200,
    "Copenhagen": 205,
    "TelAviv": 220,
    "Stockholm": 230,
    "LosAngeles": 230,
    "NewYork": 240,
    "Tokyo": 250,
    "Oslo": 280,
    "Moscow": 300
}

ticketssold = {
    'Monday': 0.5,
    'Tuesday': 0.6,
    'Wednesday': 0.7,
    'Thursday': 0.8,
    'Friday': 0.9,
    'Saturday': 0.9,
    'Sunday': 0.7
}

## pre process data

In [159]:
df_venue_info = pd.DataFrame(venue_info[1:], columns=venue_info[0])
df_change_rate = pd.DataFrame(change_rate[1:], columns=change_rate[0])

df_venue_info['currency'] = df_venue_info['Averageticketprice'].str[-3:]
df_venue_info['price(local)'] = df_venue_info['Averageticketprice'].str[:-3].astype(float)

df_venue_info = df_venue_info.merge(df_change_rate, left_on='currency',right_on='Symbol', how='left').fillna(1)
df_venue_info['price(GBP)'] = df_venue_info['price(local)'] / df_venue_info['Rate(1GBP)'].astype(float)

df_venue_info['price(GBP)'] = df_venue_info['price(GBP)'].round(3)

df_venue = df_venue_info[['City','Venuesize', 'price(GBP)']]

In [160]:
df_flight_info = pd.DataFrame(flight_info[1:], columns=flight_info[0]) 
df_hotel = pd.DataFrame(hotel_info.items(), columns=["City", "Hotel"])

df_cityinfo = df_venue.merge(df_flight_info, left_on='City', right_on='City', how='left').merge(df_hotel, left_on='City', right_on='City', how='left')

In [533]:
df_cityinfo

,City,Venuesize,price(GBP),AirportCode,TimeDifference(GMT+/-),Hotel
0,Amsterdam,900,29.167,AMS,1,170
1,Bangkok,1000,18.182,BKK,7,110
2,Barcelona,1400,25.000,BCN,1,185
3,Beijing,4000,16.854,PEK,8,110
4,Berlin,1650,20.000,BER,1,160
5,BuenosAires,800,20.000,EZE,-3,100
6,CapeTown,2250,17.391,CPT,2,100
7,Copenhagen,1550,40.698,CPH,1,205
8,Delhi,2500,20.000,DEL,5.5,95
9,Dubai,1650,35.556,DXB,4,145


In [510]:
df_time = pd.DataFrame(time[1:], columns=time[0]).set_index("").replace("", np.nan).astype(float).fillna(100)
df_cost = pd.DataFrame(cost[1:], columns=cost[0]).set_index("").replace("", np.nan).astype(float).fillna(100000)


## model

In [511]:
# Define data
cities = range(len(df_cityinfo['AirportCode']))
num_days = 8
num_people = 10

accommodation_cost = df_cityinfo.set_index('AirportCode')['Hotel'].to_dict()

flight_cost = {}

# Iterate over DataFrame to populate the dictionary
for from_city in df_cost.columns:
    for to_city in df_cost.index:
        if from_city != to_city:  # Avoid self-loops if needed
            flight_cost[(to_city, from_city)] = df_cost.loc[to_city, from_city]

flight_time = {}
for from_city in df_time.columns:
    for to_city in df_time.index:
        if from_city != to_city:  # Avoid self-loops if needed
            flight_time[(from_city, to_city)] = df_time.loc[from_city, to_city]

venuesize = df_cityinfo.set_index('AirportCode')['Venuesize'].astype(int).to_dict()
price = df_cityinfo.set_index('AirportCode')['price(GBP)'].to_dict()
cities = df_cityinfo['AirportCode'].to_list()

In [525]:
from pyomo.environ import *
from amplpy import modules

# Define data

num_days = 8
num_people = 10
num_cities = len(cities)

# Create the model
model = ConcreteModel()

# Sets
model.days = RangeSet(num_days) # 1 - 8
model.city_indices = RangeSet(num_cities) # 1 - 30

# Variables
model.route = Var(model.days, model.city_indices, domain=Binary)
model.perform = Var(model.days, domain=Binary)
# fix 
model.perform[1].fix(0)

# Objective
def profit_rule(model):
    profit = sum(model.perform[t] * venuesize[cities[c-1]] * price[cities[c-1]] * list(ticketssold.values())[t-2] * model.route[t, c] for t in range(2, num_days+1) for c in model.city_indices) # 2-8 days
    costs = sum(num_people * accommodation_cost[cities[c-1]] * model.route[t, c] for t in range(2, num_days+1) for c in model.city_indices)
    flight_costs = sum(num_people*model.route[t-1, i] * model.route[t, j] * flight_cost.get((cities[i-1], cities[j-1]),100000)
        for t in range(2, num_days+1) for i in model.city_indices for j in model.city_indices)
    return profit - costs  - flight_costs
model.profit = Objective(rule=profit_rule, sense=maximize)

# Constraints
def start_in_london_rule(model):
    return model.route[1, cities.index('LHR') + 1] == 1
model.start_in_london = Constraint(rule=start_in_london_rule)

def one_city_per_day_rule(model, t):
    return sum(model.route[t, c] for c in model.city_indices) == 1
model.one_city_per_day = Constraint(model.days, rule=one_city_per_day_rule)

def travel_rule(model, t):
    if t == 1:
        return Constraint.Skip
    return sum(model.route[t-1, i] * model.route[t, j] for i in model.city_indices for j in model.city_indices) == 1
model.travel_rule = Constraint(RangeSet(2, num_days), rule=travel_rule)

def unique_city_rule(model, c):
    return sum(model.route[t, c] for t in model.days) <= 1
model.unique_city = Constraint(model.city_indices, rule=unique_city_rule)

def rest_after_long_flight_rule(model, t):
    if t == 1:
        return Constraint.Skip
    return sum(model.route[t-1, i] * model.route[t, j] * flight_time.get((cities[i-1], cities[j-1]),10) for i in model.city_indices for j in model.city_indices) <= 4.0
model.rest_after_long_flight = Constraint(RangeSet(2, num_days), rule=rest_after_long_flight_rule)

# Solve the model
solver = SolverFactory(modules.find('CBC'), solver_io='nl')
result = solver.solve(model)

city_coute = []
# Print results
for t in model.days:
    for c in model.city_indices:
        if model.route[t, c].value == 1:
            city = cities[c-1]
            city_coute.append(city)
            perform = 'Yes' if model.perform[t].value == 1 else 'No'
            print(f"Day {t}: {city}, Perform: {perform}")
print(f"Total Profit: {model.profit()}")

Day 1: LHR, Perform: No
Day 2: CDG, Perform: Yes
Day 3: HEL, Perform: Yes
Day 4: CPH, Perform: Yes
Day 5: FCO, Perform: Yes
Day 6: TLV, Perform: Yes
Day 7: DXB, Perform: Yes
Day 8: DEL, Perform: Yes
Total Profit: 278056.45999999996


In [530]:

i = cities.index('LHR') +1
j = cities.index('CDG') +1
# for t in range(2, num_days+1):
#     for c in range(1, num_cities+1):
#         profit = model.perform[t].value * venuesize[cities[c-1]] * price[cities[c-1]] * list(ticketssold.values())[t-2] * model.route[t, c].value
#         if profit != 0:
#             print(f"Day {t}: {cities[c-1]}, Profit: {profit}")

#         costs = num_people * accommodation_cost[cities[c-1]] * model.route[t, c].value
#         if costs != 0:
#             print(f"Day {t}: {cities[c-1]}, Costs: {costs}")

for t in range(2, num_days+1):
    for i in model.city_indices:
        for j in model.city_indices:
            flight_costs = model.route[t-1, i].value * model.route[t, j].value * flight_cost.get((cities[i-1], cities[j-1]),100000)
            if flight_costs != 0:
                print(f"Day {t}: {cities[j-1]}, Flight Costs: {flight_costs}")

Day 2: CDG, Flight Costs: 70.0
Day 3: HEL, Flight Costs: 90.0
Day 4: CPH, Flight Costs: 40.0
Day 5: FCO, Flight Costs: 40.0
Day 6: TLV, Flight Costs: 240.0
Day 7: DXB, Flight Costs: 160.0
Day 8: DEL, Flight Costs: 280.0


In [483]:
# model.pprint()

## post processing

In [526]:
# city_coute = ['LHR', 'CDG', 'BCN', 'BER', 'CPH', 'AMS', 'HEL', 'LHR'] # team solution
city_coute

['LHR', 'CDG', 'HEL', 'CPH', 'FCO', 'TLV', 'DXB', 'DEL']

In [537]:
df_result = pd.DataFrame(city_coute[1:], columns=['AirportCode'])

df_result = df_result.merge(df_cityinfo[['AirportCode','City']], left_on='AirportCode', right_on='AirportCode')

df_result['Flight cost (GBP)'] = [flight_cost.get((city_coute[i-1], city_coute[i])) for i in range(1, len(city_coute))]
df_result['Hotel cost (GBP)'] = [accommodation_cost[city] for city in city_coute[1:]]
df_result['time (hours)'] = [flight_time.get((city_coute[i-1], city_coute[i])) for i in range(1, len(city_coute))]
df_result['Total cost (GBP)'] = (df_result['Flight cost (GBP)'] + df_result['Hotel cost (GBP)'] )* num_people

df_result['ticket price (GBP)'] = df_cityinfo.set_index('AirportCode').loc[city_coute[1:]]['price(GBP)'].to_list()
df_result['Venuesize'] = df_cityinfo.set_index('AirportCode').loc[city_coute[1:]]['Venuesize'].astype(int).to_list()
df_result['ticket sold'] = ticketssold.values()
df_result['Total revenue (GBP)'] = df_result['ticket price (GBP)'] * df_result['Venuesize'] * df_result['ticket sold']
df_result['Profit (GBP)'] = df_result['Total revenue (GBP)'] - df_result['Total cost (GBP)']

df_result

,AirportCode,City,Flight cost (GBP),Hotel cost (GBP),time (hours),Total cost (GBP),ticket price (GBP),Venuesize,ticket sold,Total revenue (GBP),Profit (GBP)
0,CDG,Paris,70.0,180,1.0,2500.0,25.000,1800,0.5,22500.00,20000.00
1,HEL,Helsinki,90.0,180,3.0,2700.0,32.500,2100,0.6,40950.00,38250.00
2,CPH,Copenhagen,40.0,205,2.0,2450.0,40.698,1550,0.7,44157.33,41707.33
3,FCO,Rome,40.0,150,3.0,1900.0,20.833,1850,0.8,30832.84,28932.84
4,TLV,TelAviv,240.0,220,3.5,4600.0,42.553,1900,0.9,72765.63,68165.63
5,DXB,Dubai,160.0,145,3.0,3050.0,35.556,1650,0.9,52800.66,49750.66
6,DEL,Delhi,280.0,95,4.0,3750.0,20.000,2500,0.7,35000.00,31250.00


In [538]:
df_result.to_excel('data/result.xlsx', index=False)

In [528]:
df_result['Profit (GBP)'].sum()

278056.45999999996